# 1. Load cropped

In [1]:
import os
import glob
import pandas as pd
import random
from PIL import Image
import torch
import torchvision.transforms as transforms
from torch.utils import data
import numpy as np

In [68]:
import os
import glob
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data

# Paso 1: Definir la función load_cropped
def load_cropped(folder_path, csv_path, patient_list, sample_size=200):
    # Cargar el CSV con pandas
    df = pd.read_csv(csv_path)
    
    # Convertir el CSV a un diccionario para un acceso rápido
    patient_metadata = {row['CODI']: row['DENSITAT'] for _, row in df.iterrows()}

    # Inicializar la estructura de datos para almacenar los datos de los pacientes seleccionados
    patients_data = []
    images_list = []
    
    # Iterar sobre cada paciente en la lista de IDs proporcionada
    for patient_id in patient_list:
        # Obtener carpeta del paciente
        patient_folder = glob.glob(os.path.join(folder_path, f"{patient_id}_*"))[0]

        # Verificar que el paciente esté en el CSV
        if patient_id in patient_metadata:
            # Obtener todas las imágenes .png dentro de la carpeta del paciente
            images = glob.glob(os.path.join(patient_folder, "*.png"))
            
            # Si el paciente tiene imágenes en su carpeta
            if images:
                # Mezclar la lista de imágenes
                random.shuffle(images)
                
                # Seleccionar una muestra de tamaño sample_size o menos si hay menos imágenes
                images_sampled = random.sample(images, min(sample_size, len(images)))
                
                for image_path in images_sampled:
                    # Cargar la imagen
                    image = Image.open(image_path)
                    
                    # Convertir a numpy array
                    image_np = np.array(image)
                    
                    # Añadir la imagen a la lista de imágenes
                    images_list.append(image_np)

                # Binariar densidad
                if patient_metadata[patient_id] == "NEGATIVA":
                    dens = 0
                else:
                    dens = 1
                
                # Añadir la densidad a la lista de metadatos
                patients_data.extend([dens] * len(images_sampled))
    
    return images_list, patients_data

# Paso 2: Crear la clase Standard_Dataset
class Standard_Dataset(data.Dataset):
    def __init__(self, X, Y=None, transformation=None):
        super().__init__()
        self.X = X
        self.y = Y
        self.transformation = transformation
 
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        if self.transformation:
            image = self.transformation(image)
            
        image_tensor = torch.from_numpy(image).float()
        
        if self.y is not None:
            label = torch.tensor(self.y[idx], dtype=torch.float32)
            return image_tensor, label
        else:
            return image_tensor

# Paso 3: Cargar los datos
folder_path = "C:/Users/mirvi/Desktop/mii/UAB/4.1/PSIV2/detect mateicules/repte3/psiv-repte3/data/Cropped_sample"
csv_path = "C:/Users/mirvi/Desktop/mii/UAB/4.1/PSIV2/detect mateicules/repte3/psiv-repte3/data/PatientDiagnosis.csv"
patient_list = ["B22-25", "B22-29"]
sample_size = 200

images, labels = load_cropped(folder_path, csv_path, patient_list, sample_size)

# Paso 4: Crear el Dataset y DataLoader
dataset = Standard_Dataset(images, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Paso 5: Probar el DataLoader
for batch_idx, (data, target) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}")
    print("Datos (imágenes):", data.shape)
    print("Etiquetas (densidad):", target)
    if batch_idx == 0:
        break  # Solo mostrar el primer batch para verificar que todo funciona


Batch 1
Datos (imágenes): torch.Size([16, 256, 256, 4])
Etiquetas (densidad): tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
